In [1]:
## Bring in needed mods
import glob, pandas as pd, scipy.stats as ss, numpy as np

## Load in matplotlib for ploting
from matplotlib import pyplot as plt

## Load in ftns from epi genome vis
from epigenomevisulization import *

In [2]:
## Load in encode, data names, set path
data_name_path = '../DATA/ENCODE/data.names.txt'

## Load
data_names = pd.read_csv(data_name_path,sep=' ',comment='#')

## Reset names to the three columns of interest
data_names = data_names[['Cellline','Experiment','Replicate']]

## Print to page
data_names

,Cellline,Experiment,Replicate
0,A549,ENCSR032RGS,ENCLB404SKN
1,A549,ENCSR032RGS,ENCLB605LCC
2,A549,ENCSR032RGS,ENCLB817BKI
3,HepG2,ENCSR042AWH,ENCLB074EQT
4,HepG2,ENCSR042AWH,ENCLB324GIU
5,RWPE2,ENCSR080SNF,ENCLB293SLX
6,RWPE2,ENCSR080SNF,ENCLB734LAL
7,RWPE2,ENCSR080SNF,ENCLB984XHJ
8,GM12878,ENCSR095QNB,ENCLB584REF
9,GM12878,ENCSR095QNB,ENCLB907YRF


In [3]:
## Set paths to A549, mapped to the T2T reference
mapping_stats_t2t_wc = '../DATA/A549/mapping_stats/merged.*.txt'

## gather paths
mapping_stats_t2t_paths = sortglob(mapping_stats_t2t_wc)

## Format the stats
t2t_stats = formatdfs(mapping_stats_t2t_paths)

## Reset the index 
t2t_stats.reset_index(inplace=True)

## Set the replicate 
t2t_stats['Replicate'] = t2t_stats['index']

## Drop the index 
t2t_stats.drop(['index'],axis=1,inplace=True)

## Print dataframe
t2t_stats

Mapping,Total,filtq30,notused,dups,chrM,unmapped,placed,lowqual,Replicate
0,65405524,23987725,2973344,3093813,33653170,45420,3486,1648566,2501_001
1,84816540,22605005,2595465,2481489,55231224,30232,2118,1871007,2501_002
2,64084756,17618743,2122339,1979951,40809830,66300,1172,1486421,2501_003
3,133625408,35069198,4386111,4418826,86279921,31955,NaN,3439397,2501_007
4,69273610,15377297,1834370,1780806,48556785,27699,NaN,1696653,2501_008
5,86963986,42567716,4788405,5703777,31620108,117296,3732,2162952,2501_018
6,84297712,28744542,3582876,5400961,44737775,115586,3034,1712938,2501_019
7,97877188,35836016,4491769,5663997,49816243,38764,3782,2026617,2501_020


In [4]:
## Load in the logs 
t2t_logs = sortglob('../DATA/A549/mapping_logs/slurpy.*.log.txt')

## Get the loci 
t2t_loci = getloci(t2t_logs)

## Show the loci 
t2t_loci

,Replicate,Loci,FrIP
0,2501_001,110323,0.5880
1,2501_002,81917,0.3404
2,2501_003,82496,0.3702
3,2501_007,90386,0.3515
4,2501_008,79933,0.4202
5,2501_018,130475,0.6360
6,2501_019,107737,0.6391
7,2501_020,117087,0.6595


In [5]:
## Set the replicate group 
rep_group =  [0,0,0,1,1,2,2,2]

## Format the names dataframes for our A549 data
our_names = pd.DataFrame([['A549' for i in rep_group],
                          ['This study %s'%rep_group[i] for i in range(len(mapping_stats_t2t_paths))],
                          [i for i in t2t_stats.Replicate]]).T

## Set the column names 
our_names.columns = data_names.columns

## Set the replciate columsn 
#our_names['Replicate'] = [ '\_'.join(a.split('_')) for a in  our_names.Replicate.tolist()]

## Print the names dataframe
our_names

,Cellline,Experiment,Replicate
0,A549,This study 0,2501_001
1,A549,This study 0,2501_002
2,A549,This study 0,2501_003
3,A549,This study 1,2501_007
4,A549,This study 1,2501_008
5,A549,This study 2,2501_018
6,A549,This study 2,2501_019
7,A549,This study 2,2501_020


In [6]:
## Merge dataframes
t2t_a549 = our_names.merge(t2t_stats).merge(t2t_loci).copy()

## Replace the missing values
t2t_a549.replace(np.nan,0,inplace=True)

## Convert all the 4th over columns to int
for c in t2t_a549.columns[3:-1]:
    t2t_a549[c] = t2t_a549[c].apply(int)
    
## Moved the placed to the unmapped column
t2t_a549['unmapped'] = t2t_a549['unmapped'].values + t2t_a549['placed'].values

## Drop the placed
t2t_a549.drop('placed',axis=1,inplace=True)

## Print to page 
t2t_a549

,Cellline,Experiment,Replicate,Total,filtq30,notused,dups,chrM,unmapped,lowqual,Loci,FrIP
0,A549,This study 0,2501_001,65405524,23987725,2973344,3093813,33653170,48906,1648566,110323,0.5880
1,A549,This study 0,2501_002,84816540,22605005,2595465,2481489,55231224,32350,1871007,81917,0.3404
2,A549,This study 0,2501_003,64084756,17618743,2122339,1979951,40809830,67472,1486421,82496,0.3702
3,A549,This study 1,2501_007,133625408,35069198,4386111,4418826,86279921,31955,3439397,90386,0.3515
4,A549,This study 1,2501_008,69273610,15377297,1834370,1780806,48556785,27699,1696653,79933,0.4202
5,A549,This study 2,2501_018,86963986,42567716,4788405,5703777,31620108,121028,2162952,130475,0.6360
6,A549,This study 2,2501_019,84297712,28744542,3582876,5400961,44737775,118620,1712938,107737,0.6391
7,A549,This study 2,2501_020,97877188,35836016,4491769,5663997,49816243,42546,2026617,117087,0.6595


In [7]:
## Check our work, set the error message 
read_count_err = "ERROR: Error in totaling read counts!"

## Assert this sum is zero, print error otherwise 
assert np.sum(t2t_a549[t2t_a549.columns[4:-2]].T.sum().values - t2t_a549['Total']) == 0, read_count_err

In [8]:
## Load in the encode mapping logs 
encode_logs = sortglob('../DATA/ENCODE/logs/slurpy.*.log.txt')

## View the frist three 
encode_logs[:3]

['../DATA/ENCODE/logs/slurpy.ENCLB074EQT.log.txt',
 '../DATA/ENCODE/logs/slurpy.ENCLB293SLX.log.txt',
 '../DATA/ENCODE/logs/slurpy.ENCLB324GIU.log.txt']

In [9]:
encode_loci = getloci(encode_logs)
encode_loci.head()

,Replicate,Loci,FrIP
0,ENCLB074EQT,173756,0.4257
1,ENCLB293SLX,166239,0.4740
2,ENCLB324GIU,135767,0.4605
3,ENCLB404SKN,201532,0.5898
4,ENCLB432QLN,178156,0.5363


In [10]:
## Set paths the the stats dir
ms_encode_t2t_paths = sortglob('../DATA/ENCODE/stats/*.txt')

## Load in and format 
ms_encode_t2t = formatdfs(ms_encode_t2t_paths)

## Reset the index 
ms_encode_t2t.reset_index(inplace=True)

## Rename the index to replicate 
ms_encode_t2t['Replicate'] = ms_encode_t2t['index']

## Drop index 
ms_encode_t2t.drop(['index'],axis=1,inplace=True)

## Merge dataframes 
ms_encode_t2t = data_names.merge(ms_encode_t2t).merge(encode_loci)

## Iterate thru the columns and convert to int
for c in ms_encode_t2t.columns.tolist()[3:-1]:
    ms_encode_t2t[c] = ms_encode_t2t[c].apply(int)

## View head 
ms_encode_t2t

,Cellline,Experiment,Replicate,Total,filtq30,notused,chrM,dups,unmapped,lowqual,Loci,FrIP
0,A549,ENCSR032RGS,ENCLB404SKN,341325836,259029456,21246814,12009324,46948944,408486,1682812,201532,0.5898
1,A549,ENCSR032RGS,ENCLB605LCC,442074976,329679445,27536117,15475857,66338506,506856,2538195,194975,0.5994
2,A549,ENCSR032RGS,ENCLB817BKI,277970512,211291691,18456829,11170486,35323112,343051,1385343,206536,0.5596
3,HepG2,ENCSR042AWH,ENCLB074EQT,76077306,48113686,6037783,8348893,11668633,235020,1673291,173756,0.4257
4,HepG2,ENCSR042AWH,ENCLB324GIU,88838406,48246610,6580203,19207768,12021756,605060,2177009,135767,0.4605
5,RWPE2,ENCSR080SNF,ENCLB293SLX,67263926,55152003,6718663,753542,2685741,286519,1667458,166239,0.4740
6,RWPE2,ENCSR080SNF,ENCLB734LAL,53441754,43166947,5244472,1277887,2088775,323207,1340466,177496,0.4555
7,RWPE2,ENCSR080SNF,ENCLB984XHJ,60212304,48162285,5946340,1888964,2288274,331284,1595157,154758,0.4652
8,GM12878,ENCSR095QNB,ENCLB584REF,76479882,46889870,4260513,11245729,12635046,252057,1196667,114746,0.7159
9,GM12878,ENCSR095QNB,ENCLB907YRF,69456510,49588811,4319318,7334534,6740176,186878,1286793,134743,0.6452


In [11]:
## Check our wrok, assert this sum is zero 
assert np.sum(ms_encode_t2t[ms_encode_t2t.columns[4:-2]].T.sum().values - ms_encode_t2t['Total'].values) == 0, read_count_err

In [12]:
## Merge the table 
tabel_all = pd.concat([t2t_a549,ms_encode_t2t]).reset_index(drop=True)

## Make a copy of replicate name 
rep_names = tabel_all['Replicate']

## Initlize list 
new_rep_names = []

## Rename
for r in rep_names:
    if '_' in r:
        new_name = '\_'.join(r.split('_'))
    else:
        new_name = r
    new_rep_names.append(new_name)
    
## reassign
tabel_all['Replicate'] = new_rep_names

## Set a tmp sample name
tabel_all['Sample Title'] = 'aaa'

## Group by replicates
by_rep = tabel_all.groupby(['Cellline','Experiment']).count().reset_index()[tabel_all.columns[:3]]

## Iteratte thru the rows for the by rep table 
for i,row in by_rep.iterrows():
    
    ## Parse off a part of the table 
    tmp = tabel_all[(tabel_all.Experiment==row.Experiment) & (tabel_all.Cellline==row.Cellline)]
    
    ## Iterate thru thru the row replicates 
    for j in range(row.Replicate):

        ## Rename the sample
        tabel_all.loc[tmp.index[j],'Sample Title'] = '%s$_{%s0%s}$'%(row.Cellline,i,j)
        
## Set a scourse column
source = []

## Iteraate thru the rows 
for i,s in tabel_all.iterrows():

    ## Get the original expeirment name
    exp_source = s.Experiment
    
    ## Patch the source 
    if 'This study' in exp_source:
        new_source = 'This study'
    else:
        new_source = exp_source
        
    ## Append the sources 
    source.append(new_source)
    
## Append the the column
tabel_all['Source'] = source

## Drop the old experiment column
tabel_all.drop('Experiment',axis=1,inplace=True)

## Set column dict
column_dict = {'Cellline':'Cell Line','Loci':'MACS2 Peaks','dups':'Duplicates',
               'chrM':'mtDNA','filtq30':'Mapped Reads','Total':'Total Reads',
               'Replicate':'Replicate Name','notused':'Not Used','unmapped':'Un-mapped', 
               'lowqual':'Low Quality Reads'}

## Rename columns
tabel_all.rename(columns=column_dict,inplace=True)

## Sort values by sample title
tabel_all.sort_values('Sample Title',inplace=True)

## Set the save path
full_savepath = '../DATA/full.table.csv'

## Save out the tabel
tabel_all.to_csv(full_savepath,index=False)

## print to screen
tabel_all

,Cell Line,Replicate Name,Total Reads,Mapped Reads,Not Used,Duplicates,mtDNA,Un-mapped,Low Quality Reads,MACS2 Peaks,FrIP,Sample Title,Source
8,A549,ENCLB404SKN,341325836,259029456,21246814,46948944,12009324,408486,1682812,201532,0.5898,A549$_{000}$,ENCSR032RGS
9,A549,ENCLB605LCC,442074976,329679445,27536117,66338506,15475857,506856,2538195,194975,0.5994,A549$_{001}$,ENCSR032RGS
10,A549,ENCLB817BKI,277970512,211291691,18456829,35323112,11170486,343051,1385343,206536,0.5596,A549$_{002}$,ENCSR032RGS
0,A549,2501\_001,65405524,23987725,2973344,3093813,33653170,48906,1648566,110323,0.5880,A549$_{100}$,This study
1,A549,2501\_002,84816540,22605005,2595465,2481489,55231224,32350,1871007,81917,0.3404,A549$_{101}$,This study
2,A549,2501\_003,64084756,17618743,2122339,1979951,40809830,67472,1486421,82496,0.3702,A549$_{102}$,This study
3,A549,2501\_007,133625408,35069198,4386111,4418826,86279921,31955,3439397,90386,0.3515,A549$_{200}$,This study
4,A549,2501\_008,69273610,15377297,1834370,1780806,48556785,27699,1696653,79933,0.4202,A549$_{201}$,This study
5,A549,2501\_018,86963986,42567716,4788405,5703777,31620108,121028,2162952,130475,0.6360,A549$_{300}$,This study
6,A549,2501\_019,84297712,28744542,3582876,5400961,44737775,118620,1712938,107737,0.6391,A549$_{301}$,This study


In [13]:
## Set the columns of S. table 1
s_table1_cols =  ['Sample Title','Cell Line','Total Reads','Mapped Reads',
                  'Not Used','mtDNA','Duplicates','Un-mapped','Low Quality Reads',
                  'Replicate Name','Source']

## Set save path for the S table 1
s_table1_savepath = '../DATA/S.table.1.csv'

## Split off some of the tabel
Stable1 = tabel_all[s_table1_cols]

## Save out the csv
Stable1.to_csv(s_table1_savepath,index=False)

## Print the tabel
Stable1

,Sample Title,Cell Line,Total Reads,Mapped Reads,Not Used,mtDNA,Duplicates,Un-mapped,Low Quality Reads,Replicate Name,Source
8,A549$_{000}$,A549,341325836,259029456,21246814,12009324,46948944,408486,1682812,ENCLB404SKN,ENCSR032RGS
9,A549$_{001}$,A549,442074976,329679445,27536117,15475857,66338506,506856,2538195,ENCLB605LCC,ENCSR032RGS
10,A549$_{002}$,A549,277970512,211291691,18456829,11170486,35323112,343051,1385343,ENCLB817BKI,ENCSR032RGS
0,A549$_{100}$,A549,65405524,23987725,2973344,33653170,3093813,48906,1648566,2501\_001,This study
1,A549$_{101}$,A549,84816540,22605005,2595465,55231224,2481489,32350,1871007,2501\_002,This study
2,A549$_{102}$,A549,64084756,17618743,2122339,40809830,1979951,67472,1486421,2501\_003,This study
3,A549$_{200}$,A549,133625408,35069198,4386111,86279921,4418826,31955,3439397,2501\_007,This study
4,A549$_{201}$,A549,69273610,15377297,1834370,48556785,1780806,27699,1696653,2501\_008,This study
5,A549$_{300}$,A549,86963986,42567716,4788405,31620108,5703777,121028,2162952,2501\_018,This study
6,A549$_{301}$,A549,84297712,28744542,3582876,44737775,5400961,118620,1712938,2501\_019,This study


In [14]:
## Set table 1 columns
table1_cols = ['Sample Title','Cell Line','Mapped Reads','MACS2 Peaks','FrIP','Source']

## Split off some of the tabel
table1 = tabel_all[table1_cols]

## Set the table savepath
table1_savepath = "../DATA/table.1.csv"

## Save out the csv
table1.to_csv(table1_savepath,index=False)

## Print the tabel
table1

,Sample Title,Cell Line,Mapped Reads,MACS2 Peaks,FrIP,Source
8,A549$_{000}$,A549,259029456,201532,0.5898,ENCSR032RGS
9,A549$_{001}$,A549,329679445,194975,0.5994,ENCSR032RGS
10,A549$_{002}$,A549,211291691,206536,0.5596,ENCSR032RGS
0,A549$_{100}$,A549,23987725,110323,0.5880,This study
1,A549$_{101}$,A549,22605005,81917,0.3404,This study
2,A549$_{102}$,A549,17618743,82496,0.3702,This study
3,A549$_{200}$,A549,35069198,90386,0.3515,This study
4,A549$_{201}$,A549,15377297,79933,0.4202,This study
5,A549$_{300}$,A549,42567716,130475,0.6360,This study
6,A549$_{301}$,A549,28744542,107737,0.6391,This study


In [15]:
## Check we have all the columns across the two tables
## Set the error message
miss_col_err = "Error we are missing columns!"

## Assert the lenths are the same 
assert len(set(table1_cols + s_table1_cols) & set(tabel_all.columns.tolist())) == len(tabel_all.columns), miss_col_err